In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import re

In [3]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

df = pd.read_csv('2021 END OF YEAR ROUNDUP_ TOP RESIDENTIAL SALES IN SOUTH FLORIDA - FINAL.csv')
df.drop('SELLER', axis=1, inplace=True)
df

,Rank,ADDRESS,CITY,PRICE,SQUARE FEET,BUYER (Property Shark),SOLD DATE,MLS,Clip,Notes
0,1,535 N County Road,Palm Beach,$122.7M,NaN,King County Road Inc,NaN,NaN,https://www.palmbeachdailynews.com/story/busin...,Chris Leavitt **repped buyer
1,2,1840 South Ocean Boulevard,Palm Beach,$109.63M,"15,954","1840 South Ocean Trust, Kochman Ronald Trustee",6/14/21,PBBOR #21-378,https://therealdeal.com/miami/2021/06/17/germa...,NaN
2,3,1341 South Ocean Boulevard,Palm Beach,$95.00M,"13,936","Kochman Ronald S Trustee, 1341 South Ocean Bou...",5/14/21,NaN,https://therealdeal.com/miami/2021/05/20/punch...,NaN
3,4,2000 South Ocean Boulevard,Lake Worth,$94.17M,"20,597","2000 S Ocean Trust, Kochman Ronald Trustee",3/8/21,PBBOR #17-1227,https://therealdeal.com/miami/2021/03/09/ziff-...,NaN
4,5,12525 Seminole Beach Road/12210 Banyan Rd,North Palm Beach,$80.00M,"15,514",Florida Estates LLC,4/8/21,RAPB #RX-10632970,https://therealdeal.com/miami/2021/04/13/billi...,Also referred to as 12210 Banyan Rd
5,6,8 Star Island Drive,Miami Beach,$75.00M,"16,619",NaN,12/20/21,NaN,https://therealdeal.com/miami/2021/12/21/water...,NaN
6,7,3488 Saint Gaudens Road,Coconut Grove,$65M,NaN,CH2CG LLC,NaN,NaN,https://therealdeal.com/miami/2021/07/29/65m-s...,Oren Alexander **repped buyer
7,8,901 North Ocean Boulevard,Palm Beach,$64.00M,"15,306",Beechgreen LLC,9/2/21,PBBOR #21-281,https://therealdeal.com/miami/2021/09/01/clark...,NaN
8,9,174 Via Del Lago,Palm Beach,$57.00M,"15,241",174 Vdl Investment LLC,3/28/21,PBBOR #20-2192,https://therealdeal.com/miami/2021/03/30/hotel...,NaN
9,10,520 Island Drive,Palm Beach,$49.14M,NaN,"Miller Elizabeth R, Dinan James G (Founder, Yo...",5/18/21,PBBOR #21-627,https://www.palmbeachdailynews.com/story/busin...,NaN


In [4]:
df.columns = df.columns.str.replace('(','')
df.columns = df.columns.str.replace(')','')
df.columns = df.columns.str.replace(' ','_')
df

<ipython-input-4-f6512fdce826>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('(','')
<ipython-input-4-f6512fdce826>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(')','')


,Rank,ADDRESS,CITY,PRICE,SQUARE_FEET,BUYER_Property_Shark,SOLD_DATE,MLS,Clip,Notes
0,1,535 N County Road,Palm Beach,$122.7M,NaN,King County Road Inc,NaN,NaN,https://www.palmbeachdailynews.com/story/busin...,Chris Leavitt **repped buyer
1,2,1840 South Ocean Boulevard,Palm Beach,$109.63M,"15,954","1840 South Ocean Trust, Kochman Ronald Trustee",6/14/21,PBBOR #21-378,https://therealdeal.com/miami/2021/06/17/germa...,NaN
2,3,1341 South Ocean Boulevard,Palm Beach,$95.00M,"13,936","Kochman Ronald S Trustee, 1341 South Ocean Bou...",5/14/21,NaN,https://therealdeal.com/miami/2021/05/20/punch...,NaN
3,4,2000 South Ocean Boulevard,Lake Worth,$94.17M,"20,597","2000 S Ocean Trust, Kochman Ronald Trustee",3/8/21,PBBOR #17-1227,https://therealdeal.com/miami/2021/03/09/ziff-...,NaN
4,5,12525 Seminole Beach Road/12210 Banyan Rd,North Palm Beach,$80.00M,"15,514",Florida Estates LLC,4/8/21,RAPB #RX-10632970,https://therealdeal.com/miami/2021/04/13/billi...,Also referred to as 12210 Banyan Rd
5,6,8 Star Island Drive,Miami Beach,$75.00M,"16,619",NaN,12/20/21,NaN,https://therealdeal.com/miami/2021/12/21/water...,NaN
6,7,3488 Saint Gaudens Road,Coconut Grove,$65M,NaN,CH2CG LLC,NaN,NaN,https://therealdeal.com/miami/2021/07/29/65m-s...,Oren Alexander **repped buyer
7,8,901 North Ocean Boulevard,Palm Beach,$64.00M,"15,306",Beechgreen LLC,9/2/21,PBBOR #21-281,https://therealdeal.com/miami/2021/09/01/clark...,NaN
8,9,174 Via Del Lago,Palm Beach,$57.00M,"15,241",174 Vdl Investment LLC,3/28/21,PBBOR #20-2192,https://therealdeal.com/miami/2021/03/30/hotel...,NaN
9,10,520 Island Drive,Palm Beach,$49.14M,NaN,"Miller Elizabeth R, Dinan James G (Founder, Yo...",5/18/21,PBBOR #21-627,https://www.palmbeachdailynews.com/story/busin...,NaN


In [5]:
df['SQUARE_FEET'] = df['SQUARE_FEET'].replace(',','',regex=True)
df['SQUARE_FEET'] = pd.to_numeric(df['SQUARE_FEET'])
df.dtypes

Rank                      int64
ADDRESS                  object
CITY                     object
PRICE                    object
SQUARE_FEET             float64
BUYER_Property_Shark     object
SOLD_DATE                object
MLS                      object
Clip                     object
Notes                    object
dtype: object

In [6]:
df['geo_address'] = df['ADDRESS'] + ' ' + df['CITY'] + ' FL'
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df

,Rank,ADDRESS,CITY,PRICE,SQUARE_FEET,BUYER_Property_Shark,SOLD_DATE,MLS,Clip,Notes,geo_address
0,1,535 N County Road,Palm Beach,$122.7M,NaN,King County Road Inc,NaN,NaN,https://www.palmbeachdailynews.com/story/busin...,Chris Leavitt **repped buyer,535 N County Road Palm Beach FL
1,2,1840 South Ocean Boulevard,Palm Beach,$109.63M,15954.0,"1840 South Ocean Trust, Kochman Ronald Trustee",6/14/21,PBBOR #21-378,https://therealdeal.com/miami/2021/06/17/germa...,NaN,1840 South Ocean Boulevard Palm Beach FL
2,3,1341 South Ocean Boulevard,Palm Beach,$95.00M,13936.0,"Kochman Ronald S Trustee, 1341 South Ocean Bou...",5/14/21,NaN,https://therealdeal.com/miami/2021/05/20/punch...,NaN,1341 South Ocean Boulevard Palm Beach FL
3,4,2000 South Ocean Boulevard,Lake Worth,$94.17M,20597.0,"2000 S Ocean Trust, Kochman Ronald Trustee",3/8/21,PBBOR #17-1227,https://therealdeal.com/miami/2021/03/09/ziff-...,NaN,2000 South Ocean Boulevard Lake Worth FL
4,5,12525 Seminole Beach Road/12210 Banyan Rd,North Palm Beach,$80.00M,15514.0,Florida Estates LLC,4/8/21,RAPB #RX-10632970,https://therealdeal.com/miami/2021/04/13/billi...,Also referred to as 12210 Banyan Rd,12525 Seminole Beach Road/12210 Banyan Rd Nort...
5,6,8 Star Island Drive,Miami Beach,$75.00M,16619.0,NaN,12/20/21,NaN,https://therealdeal.com/miami/2021/12/21/water...,NaN,8 Star Island Drive Miami Beach FL
6,7,3488 Saint Gaudens Road,Coconut Grove,$65M,NaN,CH2CG LLC,NaN,NaN,https://therealdeal.com/miami/2021/07/29/65m-s...,Oren Alexander **repped buyer,3488 Saint Gaudens Road Coconut Grove FL
7,8,901 North Ocean Boulevard,Palm Beach,$64.00M,15306.0,Beechgreen LLC,9/2/21,PBBOR #21-281,https://therealdeal.com/miami/2021/09/01/clark...,NaN,901 North Ocean Boulevard Palm Beach FL
8,9,174 Via Del Lago,Palm Beach,$57.00M,15241.0,174 Vdl Investment LLC,3/28/21,PBBOR #20-2192,https://therealdeal.com/miami/2021/03/30/hotel...,NaN,174 Via Del Lago Palm Beach FL
9,10,520 Island Drive,Palm Beach,$49.14M,NaN,"Miller Elizabeth R, Dinan James G (Founder, Yo...",5/18/21,PBBOR #21-627,https://www.palmbeachdailynews.com/story/busin...,NaN,520 Island Drive Palm Beach FL


In [121]:
#df.at[0,'Property Address']=('1111 North Ocean Drive, Hollywood, FL')

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)

In [9]:
df.head(1)

,Rank,ADDRESS,CITY,PRICE,SQUARE_FEET,BUYER_Property_Shark,SOLD_DATE,MLS,Clip,Notes,geo_address,loc
0,1,535 N County Road,Palm Beach,$122.7M,NaN,King County Road Inc,NaN,NaN,https://www.palmbeachdailynews.com/story/busin...,Chris Leavitt **repped buyer,535 N County Road Palm Beach FL,"(535 N County Rd, Palm Beach, FL 33480, USA, (..."


In [10]:
df['loc']

0    (535 N County Rd, Palm Beach, FL 33480, USA, (...
1    (1840 S Ocean Blvd, Palm Beach, FL 33480, USA,...
2    (1341 S Ocean Blvd, Palm Beach, FL 33480, USA,...
3    (2000 S Ocean Blvd, Palm Beach, FL 33480, USA,...
4    (12525 Seminole Beach Rd, North Palm Beach, FL...
5    (8 W Star Island Dr, Miami Beach, FL 33139, US...
6    (3488 St Gaudens Rd, Miami, FL 33133, USA, (25...
7    (901 N Ocean Blvd, Palm Beach, FL 33480, USA, ...
8    (174 Via Del Lago, Palm Beach, FL 33480, USA, ...
9    (520 Island Dr, Palm Beach, FL 33480, USA, (26...
Name: loc, dtype: object

In [11]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [12]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,Rank,ADDRESS,CITY,PRICE,SQUARE_FEET,BUYER_Property_Shark,SOLD_DATE,MLS,Clip,Notes,geo_address,loc,point,lat,lon,altitude
0,1,535 N County Road,Palm Beach,$122.7M,NaN,King County Road Inc,NaN,NaN,https://www.palmbeachdailynews.com/story/busin...,Chris Leavitt **repped buyer,535 N County Road Palm Beach FL,"(535 N County Rd, Palm Beach, FL 33480, USA, (...","(26.7327407, -80.0365917, 0.0)",26.732741,-80.036592,0.0
1,2,1840 South Ocean Boulevard,Palm Beach,$109.63M,15954.0,"1840 South Ocean Trust, Kochman Ronald Trustee",6/14/21,PBBOR #21-378,https://therealdeal.com/miami/2021/06/17/germa...,NaN,1840 South Ocean Boulevard Palm Beach FL,"(1840 S Ocean Blvd, Palm Beach, FL 33480, USA,...","(26.6524428, -80.0373809, 0.0)",26.652443,-80.037381,0.0
2,3,1341 South Ocean Boulevard,Palm Beach,$95.00M,13936.0,"Kochman Ronald S Trustee, 1341 South Ocean Bou...",5/14/21,NaN,https://therealdeal.com/miami/2021/05/20/punch...,NaN,1341 South Ocean Boulevard Palm Beach FL,"(1341 S Ocean Blvd, Palm Beach, FL 33480, USA,...","(26.6673352, -80.0365949, 0.0)",26.667335,-80.036595,0.0
3,4,2000 South Ocean Boulevard,Lake Worth,$94.17M,20597.0,"2000 S Ocean Trust, Kochman Ronald Trustee",3/8/21,PBBOR #17-1227,https://therealdeal.com/miami/2021/03/09/ziff-...,NaN,2000 South Ocean Boulevard Lake Worth FL,"(2000 S Ocean Blvd, Palm Beach, FL 33480, USA,...","(26.6437781, -80.03794859999999, 0.0)",26.643778,-80.037949,0.0
4,5,12525 Seminole Beach Road/12210 Banyan Rd,North Palm Beach,$80.00M,15514.0,Florida Estates LLC,4/8/21,RAPB #RX-10632970,https://therealdeal.com/miami/2021/04/13/billi...,Also referred to as 12210 Banyan Rd,12525 Seminole Beach Road/12210 Banyan Rd Nort...,"(12525 Seminole Beach Rd, North Palm Beach, FL...","(26.856091, -80.0464028, 0.0)",26.856091,-80.046403,0.0
5,6,8 Star Island Drive,Miami Beach,$75.00M,16619.0,NaN,12/20/21,NaN,https://therealdeal.com/miami/2021/12/21/water...,NaN,8 Star Island Drive Miami Beach FL,"(8 W Star Island Dr, Miami Beach, FL 33139, US...","(25.7768898, -80.1515369, 0.0)",25.776890,-80.151537,0.0
6,7,3488 Saint Gaudens Road,Coconut Grove,$65M,NaN,CH2CG LLC,NaN,NaN,https://therealdeal.com/miami/2021/07/29/65m-s...,Oren Alexander **repped buyer,3488 Saint Gaudens Road Coconut Grove FL,"(3488 St Gaudens Rd, Miami, FL 33133, USA, (25...","(25.7153865, -80.24750929999999, 0.0)",25.715387,-80.247509,0.0
7,8,901 North Ocean Boulevard,Palm Beach,$64.00M,15306.0,Beechgreen LLC,9/2/21,PBBOR #21-281,https://therealdeal.com/miami/2021/09/01/clark...,NaN,901 North Ocean Boulevard Palm Beach FL,"(901 N Ocean Blvd, Palm Beach, FL 33480, USA, ...","(26.746267, -80.0355469, 0.0)",26.746267,-80.035547,0.0
8,9,174 Via Del Lago,Palm Beach,$57.00M,15241.0,174 Vdl Investment LLC,3/28/21,PBBOR #20-2192,https://therealdeal.com/miami/2021/03/30/hotel...,NaN,174 Via Del Lago Palm Beach FL,"(174 Via Del Lago, Palm Beach, FL 33480, USA, ...","(26.6808159, -80.0381014, 0.0)",26.680816,-80.038101,0.0
9,10,520 Island Drive,Palm Beach,$49.14M,NaN,"Miller Elizabeth R, Dinan James G (Founder, Yo...",5/18/21,PBBOR #21-627,https://www.palmbeachdailynews.com/story/busin...,NaN,520 Island Drive Palm Beach FL,"(520 Island Dr, Palm Beach, FL 33480, USA, (26...","(26.6982729, -80.04366859999999, 0.0)",26.698273,-80.043669,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [13]:
df.columns

Index(['Rank', 'ADDRESS', 'CITY', 'PRICE', 'SQUARE_FEET',
       'BUYER_Property_Shark', 'SOLD_DATE', 'MLS', 'Clip', 'Notes',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [15]:
df.fillna('N/A', inplace=True)

In [16]:
df

,Rank,ADDRESS,CITY,PRICE,SQUARE_FEET,BUYER_Property_Shark,SOLD_DATE,MLS,Clip,Notes,geo_address,loc,point,lat,lon,altitude
0,1,535 N County Road,Palm Beach,$122.7M,N/A,King County Road Inc,N/A,N/A,https://www.palmbeachdailynews.com/story/busin...,Chris Leavitt **repped buyer,535 N County Road Palm Beach FL,"(535 N County Rd, Palm Beach, FL 33480, USA, (...","(26.7327407, -80.0365917, 0.0)",26.732741,-80.036592,0.0
1,2,1840 South Ocean Boulevard,Palm Beach,$109.63M,15954.0,"1840 South Ocean Trust, Kochman Ronald Trustee",6/14/21,PBBOR #21-378,https://therealdeal.com/miami/2021/06/17/germa...,N/A,1840 South Ocean Boulevard Palm Beach FL,"(1840 S Ocean Blvd, Palm Beach, FL 33480, USA,...","(26.6524428, -80.0373809, 0.0)",26.652443,-80.037381,0.0
2,3,1341 South Ocean Boulevard,Palm Beach,$95.00M,13936.0,"Kochman Ronald S Trustee, 1341 South Ocean Bou...",5/14/21,N/A,https://therealdeal.com/miami/2021/05/20/punch...,N/A,1341 South Ocean Boulevard Palm Beach FL,"(1341 S Ocean Blvd, Palm Beach, FL 33480, USA,...","(26.6673352, -80.0365949, 0.0)",26.667335,-80.036595,0.0
3,4,2000 South Ocean Boulevard,Lake Worth,$94.17M,20597.0,"2000 S Ocean Trust, Kochman Ronald Trustee",3/8/21,PBBOR #17-1227,https://therealdeal.com/miami/2021/03/09/ziff-...,N/A,2000 South Ocean Boulevard Lake Worth FL,"(2000 S Ocean Blvd, Palm Beach, FL 33480, USA,...","(26.6437781, -80.03794859999999, 0.0)",26.643778,-80.037949,0.0
4,5,12525 Seminole Beach Road/12210 Banyan Rd,North Palm Beach,$80.00M,15514.0,Florida Estates LLC,4/8/21,RAPB #RX-10632970,https://therealdeal.com/miami/2021/04/13/billi...,Also referred to as 12210 Banyan Rd,12525 Seminole Beach Road/12210 Banyan Rd Nort...,"(12525 Seminole Beach Rd, North Palm Beach, FL...","(26.856091, -80.0464028, 0.0)",26.856091,-80.046403,0.0
5,6,8 Star Island Drive,Miami Beach,$75.00M,16619.0,N/A,12/20/21,N/A,https://therealdeal.com/miami/2021/12/21/water...,N/A,8 Star Island Drive Miami Beach FL,"(8 W Star Island Dr, Miami Beach, FL 33139, US...","(25.7768898, -80.1515369, 0.0)",25.776890,-80.151537,0.0
6,7,3488 Saint Gaudens Road,Coconut Grove,$65M,N/A,CH2CG LLC,N/A,N/A,https://therealdeal.com/miami/2021/07/29/65m-s...,Oren Alexander **repped buyer,3488 Saint Gaudens Road Coconut Grove FL,"(3488 St Gaudens Rd, Miami, FL 33133, USA, (25...","(25.7153865, -80.24750929999999, 0.0)",25.715387,-80.247509,0.0
7,8,901 North Ocean Boulevard,Palm Beach,$64.00M,15306.0,Beechgreen LLC,9/2/21,PBBOR #21-281,https://therealdeal.com/miami/2021/09/01/clark...,N/A,901 North Ocean Boulevard Palm Beach FL,"(901 N Ocean Blvd, Palm Beach, FL 33480, USA, ...","(26.746267, -80.0355469, 0.0)",26.746267,-80.035547,0.0
8,9,174 Via Del Lago,Palm Beach,$57.00M,15241.0,174 Vdl Investment LLC,3/28/21,PBBOR #20-2192,https://therealdeal.com/miami/2021/03/30/hotel...,N/A,174 Via Del Lago Palm Beach FL,"(174 Via Del Lago, Palm Beach, FL 33480, USA, ...","(26.6808159, -80.0381014, 0.0)",26.680816,-80.038101,0.0
9,10,520 Island Drive,Palm Beach,$49.14M,N/A,"Miller Elizabeth R, Dinan James G (Founder, Yo...",5/18/21,PBBOR #21-627,https://www.palmbeachdailynews.com/story/busin...,N/A,520 Island Drive Palm Beach FL,"(520 Island Dr, Palm Beach, FL 33480, USA, (26...","(26.6982729, -80.04366859999999, 0.0)",26.698273,-80.043669,0.0


In [17]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=8)
# folium.TileLayer('stamentoner').add_to(m)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'2021 South Florida Top Residential Sales')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('Palm Beach took 6 of the 10 top spots')


marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["ADDRESS"], r["CITY"], r["PRICE"], r["SQUARE_FEET"], r["BUYER_Property_Shark"], r["SOLD_DATE"], r["MLS"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Address: {bold_start}{info[0]}{bold_end}{new_line}\
    City: {bold_start}{info[1]}{bold_end}{new_line}\
    Price: {bold_start}{info[2]}{bold_end}{new_line}\
    Square Feet: {bold_start}{info[3]}{bold_end}{new_line}\
    Buyer: {bold_start}{info[4]}{bold_end}{new_line}\
    Sale Date: {bold_start}{info[5]}{bold_end}{new_line}\
    MLS: {bold_start}{info[6]}{bold_end}{new_line}'
    
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
m

In [18]:
m.save('index.html')